In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
from urllib import urlopen
from bs4 import BeautifulSoup
import os
import requests
import json
import pickle
from pandas.tseries.offsets import BDay
import datetime as dt
import re
'''this program will take around an hour to run'''

'this program will take around an hour to run'

In [2]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)      
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)    
    return tickers    

In [3]:
print "get snp500 stock tickers\n"
snp500tickers = save_sp500_tickers()

get snp500 stock tickers



In [4]:
no_hist_symbol = []

def list_of_all_stock_grading_history(good_list):
    
    all_stock_grading_history = []
    for ticker in good_list:
        print ticker
        if ticker not in no_hist_symbol:
            url = auto_symbols(ticker)
            soup = yahoo_finance_analyst_data(url)
            all_tables = grab_dict(soup)
            grading_history = upgrade_downgrade_pandas(all_tables,ticker)
            all_stock_grading_history.append(grading_history)
    concat_lists_to_pd = pd.concat(all_stock_grading_history)
    return concat_lists_to_pd

def select_ticker(tick):
    url = 'https://finance.yahoo.com/quote/{0}/analysts?p={0}'
    return url.format(tick)

def yahoo_finance_analyst_data(url):
    req = urlopen(url)
    raw = req.read()
    soup = BeautifulSoup(raw,'lxml')
    return soup

def grab_dict(soup):
    l = [i for i in soup.find_all('script')]
    mylist = [str(item) for item in l]
    m = max(mylist,key=len)
    ind = mylist.index(m)
    data = l[mylist.index(m)]
    data_dict = data.text[112:-12]
    all_tables = json.loads(data_dict)
    return all_tables

def upgrade_downgrade_pandas(dic,tick):
    try:
        stock_upgrade_downgrade_history = dic['context']['dispatcher']['stores']['QuoteSummaryStore']['upgradeDowngradeHistory']['history']
        pd_stock_upgrade_downgrade_history = pd.DataFrame(stock_upgrade_downgrade_history)
        pd_stock_upgrade_downgrade_history['epochGradeDate'] = pd.to_datetime(pd_stock_upgrade_downgrade_history['epochGradeDate'],unit='s')
        pd_stock_upgrade_downgrade_history.columns.values[1] = 'date'
        pd_stock_upgrade_downgrade_history.insert(0,'stockTicker',tick)
        return pd_stock_upgrade_downgrade_history
    except Exception:
        print "error" + tick
        no_hist_symbol.append(tick)

def auto_symbols(ticker):
    url = select_ticker(ticker)
    return url

In [5]:
print "use tickers to grab stock upgrade downgrade history\n"
stock_to_grade = list_of_all_stock_grading_history(snp500tickers)

use tickers to grab stock upgrade downgrade history

MMM
ABT
ABBV
ACN
ATVI
AYI
ADBE
AMD
AAP
AES
AET
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
AGN
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
APC
ADI
ANDV
ANSS
ANTM
AON
AOS
APA
AIV
AAPL
AMAT
APTV
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AZO
AVB
AVY
BHGE
BLL
BAC
BK
BAX
BBT
BDX
BRK.B
errorBRK.B
BBY
BIIB
BLK
HRB
BA
BKNG
BWA
BXP
BSX
BHF
BMY
AVGO
BF.B
errorBF.B
CHRW
CA
COG
CDNS
CPB
COF
CAH
KMX
CCL
CAT
CBOE
CBRE
CBS
CELG
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
COO
GLW
COST
COTY
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DLR
DFS
DISCA
DISCK
errorDISCK
DISH
DG
DLTR
D
DOV
DWDP
DPS
DTE
DRE
DUK
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EVHC
EOG
EQT
EFX
EQIX
EQR
ESS
EL
ES
RE
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FISV
FLIR
FLS
FLR
FMC
FL
F
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
GD
GE
GGP
GIS
GM
GPC
GILD
GPN
GS
GT
GWW


In [6]:
stock_to_grade['firm']= stock_to_grade['firm'].str.replace('/',"")

In [7]:
def all_hist_to_tick(df):
    list_of_stock_histories = []
    for ticker in df['stockTicker'].unique():
        try:
            stock_df = pdr.get_data_yahoo(symbols=ticker, start=df.loc[df['stockTicker']==ticker].date.iloc[-1])
            stock_df['stockTicker'] = ticker
            stock_df.reset_index(inplace = True)
            stock_df['Percent_change'] =  stock_df.Close.pct_change()
            list_of_stock_histories.append(stock_df)
            print ticker
        except Exception: 
            print 'error @ ' + ticker
    concat_lists_to_pd = pd.concat(list_of_stock_histories)
    return concat_lists_to_pd

In [8]:
print "use tickers to grab all stock price data\n"
all_snp_price_hist = all_hist_to_tick(stock_to_grade)

use tickers to grab all stock price data

MMM
ABT
ABBV
ACN
ATVI
AYI
ADBE
AMD
AAP
AES
AET
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
AGN
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
APC
ADI
ANDV
ANSS
ANTM
AON
AOS
APA
AIV
AAPL
AMAT
APTV
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AZO
AVB
AVY
BHGE
BLL
BAC
BK
BAX
BBT
BDX
BBY
BIIB
BLK
HRB
BA
BKNG
BWA
BXP
BSX
BHF
BMY
AVGO
CHRW
CA
COG
CDNS
CPB
COF
CAH
KMX
CCL
CAT
CBOE
CBRE
CBS
CELG
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
COO
GLW
COST
COTY
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DLR
DFS
DISCA
DISH
DG
DLTR
D
DOV
DWDP
DPS
DTE
DRE
DUK
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EVHC
EOG
EQT
EFX
EQIX
EQR
ESS
EL
ES
RE
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FISV
FLIR
FLS
FLR
FMC
FL
F
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
GD
GE
GGP
GIS
GM
GPC
GILD
GPN
GS
GT
GWW
HAL
HBI
HOG
HRS
HIG
HAS
HCA
HCP
HP
HSIC
HSY
HES
HPE
HLT
HOLX

In [9]:
def combine_toGrade(df):
    
    Strong_Buy = ['Outperform','Mkt Outperformer','Recomm List','Market Outperform','Mkt Outperform','NT Strong Buy','Recommended List','LT Accumulate','Outperformer','Top Pick', 'Strong Buy','Outperf. Signif.','Sector Outperform','Conviction Buy']
    Buy = ['Buy','Overweight','Trading Buy','NT Accum/LT Buy', 'NT Ntrl/LT Buy','Long-Term Buy','Above Average','LT Attractive','LT Accum','LT Buy','Accumulate','NT Accum','NT Nuet/ LT Buy','Attractive','Positive','Long-term Buy','Add','NT Buy']
    Hold = ['Hold','Neutral','Peer perform','Equal-weight','Perform-In-Line','Perform In Line','In-line','Market Perform','Mkt Performer','Average','NT Neutral','Maintain Position','ST Neutral','Sector Perform','Mixed','Market Weight','Equal-weight','Perform','Sector Weight','Peer Perform','In-Line','Fair Value','Mkt Perform','Equal Weight']
    Underperform = ['Underperform', 'Underperformer','Below Average','Unattractive','Market Underperform','Cautious','Mkt Underperform','Sector Underperform']
    Sell = ['Sell','Underweight','Reduce','Negative','Trim']

    list_dict = {
    'Strong_buy':Strong_Buy,
    'Buy':Buy,
    'Hold':Hold,
    'Underperform':Underperform,
    'Sell':Sell
    }
    reversed_dict = {val: key for key in list_dict for val in list_dict[key]}
    df['toGrade_combined'] = df['toGrade'].map(reversed_dict)
    df['fromGrade_combined'] = df['fromGrade'].map(reversed_dict)
    return df

In [10]:
print "combine like-terms of grading into a minimalized list\n"
stock_to_grade = combine_toGrade(stock_to_grade)

combine like-terms of grading into a minimalized list



In [11]:
#make copies of dataframes
stock_to_grade_copy = stock_to_grade.copy()
all_snp_price_hist_copy = all_snp_price_hist.copy()
spy_stock_hist_data = all_snp_price_hist.copy()

In [12]:
#stock_to_grade_copy['firm'] = stock_to_grade_copy['firm'].str.replace('/','')

In [13]:
all_snp_price_hist_copy.head()

,Date,Open,High,Low,Close,Adj Close,Volume,stockTicker,Percent_change
0,2012-02-09,87.540001,88.180000,87.459999,88.019997,75.432663,3031600.0,MMM,NaN
1,2012-02-10,87.199997,87.559998,86.750000,87.139999,74.678513,3085700.0,MMM,-0.009998
2,2012-02-13,87.570000,88.089996,87.160004,88.029999,75.441238,2530600.0,MMM,0.010213
3,2012-02-14,87.599998,88.000000,87.220001,87.989998,75.406952,2850300.0,MMM,-0.000454
4,2012-02-15,87.419998,87.720001,86.750000,87.010002,75.070480,3647000.0,MMM,-0.011138


In [14]:
spy_stock_hist_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,stockTicker,Percent_change
0,2012-02-09,87.540001,88.180000,87.459999,88.019997,75.432663,3031600.0,MMM,NaN
1,2012-02-10,87.199997,87.559998,86.750000,87.139999,74.678513,3085700.0,MMM,-0.009998
2,2012-02-13,87.570000,88.089996,87.160004,88.029999,75.441238,2530600.0,MMM,0.010213
3,2012-02-14,87.599998,88.000000,87.220001,87.989998,75.406952,2850300.0,MMM,-0.000454
4,2012-02-15,87.419998,87.720001,86.750000,87.010002,75.070480,3647000.0,MMM,-0.011138


In [15]:
#remove unneeded columns from the dataframes
spy_stock_hist_data = spy_stock_hist_data[['Date','stockTicker','Adj Close']]
stock_to_grade_copy = stock_to_grade_copy[['date','stockTicker','firm','toGrade_combined']]
all_snp_price_hist_copy = all_snp_price_hist_copy[['Date', 'Adj Close','stockTicker']]

In [16]:
#set the Date as index
stock_to_grade_copy=stock_to_grade_copy.set_index('date')
stock_to_grade_copy.index.names=['Date']
all_snp_price_hist_copy=all_snp_price_hist_copy.set_index('Date')

In [17]:
#change the date index to datetime object
all_snp_price_hist_copy.index.to_datetime()
stock_to_grade_copy.index.to_datetime()

DatetimeIndex(['2018-02-15', '2018-01-24', '2018-01-23', '2017-10-25',
               '2017-09-20', '2016-12-07', '2016-11-21', '2016-10-24',
               '2016-01-21', '2015-09-25',
               ...
               '2013-03-13', '2013-03-13', '2013-03-13', '2013-03-13',
               '2013-03-12', '2013-03-12', '2013-03-11', '2013-03-08',
               '2013-03-07', '2013-01-31'],
              dtype='datetime64[ns]', name=u'Date', length=21916, freq=None)

In [18]:
print "merge grading df and price df on Date and stockTicker\n"
stock_price_merge_grade = stock_to_grade_copy.set_index('stockTicker',append=True).join(all_snp_price_hist_copy.set_index('stockTicker',append=True)).reset_index(level='stockTicker')
#rearange order of columns
cols = ['Adj Close','stockTicker','firm','toGrade_combined']
stock_price_merge_grade = stock_price_merge_grade[cols]

merge grading df and price df on Date and stockTicker



In [19]:
def get_stock_data(symbols):
    for symbol in symbols:
        print symbol
        df_temp =  pdr.get_data_yahoo(symbols=symbol)
        df_temp = df_temp.rename(columns={'Adj Close':symbol})
        df_temp = df_temp[[symbol]]
        return df_temp

def use_only_spy_dates(spy,df):
    df_list = []
    for symbol in df.stockTicker.unique():
        dates = spy
        temp_df=df.loc[df['stockTicker'] == symbol]
        dates = dates.join(temp_df)
        print dates
        df_list.append(dates)
    return pd.concat(df_list)

In [20]:
print "grab dates of SPY stock ticker to serve as a constant\n"
spy_dates = get_stock_data(['SPY'])

spy_dates_symb = use_only_spy_dates(spy_dates,all_snp_price_hist_copy)

grab dates of SPY stock ticker to serve as a constant

SPY
                   SPY   Adj Close stockTicker
Date                                          
2010-01-04   95.957184         NaN         NaN
2010-01-05   96.211189         NaN         NaN
2010-01-06   96.278938         NaN         NaN
2010-01-07   96.685364         NaN         NaN
2010-01-08   97.007088         NaN         NaN
2010-01-11   97.142586         NaN         NaN
2010-01-12   96.236595         NaN         NaN
2010-01-13   97.049469         NaN         NaN
2010-01-14   97.311928         NaN         NaN
2010-01-15   96.219643         NaN         NaN
2010-01-19   97.421989         NaN         NaN
2010-01-20   96.431366         NaN         NaN
2010-01-21   94.577072         NaN         NaN
2010-01-22   92.468773         NaN         NaN
2010-01-25   92.942909         NaN         NaN
2010-01-26   92.553436         NaN         NaN
2010-01-27   92.993713         NaN         NaN
2010-01-28   91.926857         NaN         NaN
2

                   SPY  Adj Close stockTicker
Date                                         
2010-01-04   95.957184        NaN         NaN
2010-01-05   96.211189        NaN         NaN
2010-01-06   96.278938        NaN         NaN
2010-01-07   96.685364        NaN         NaN
2010-01-08   97.007088        NaN         NaN
2010-01-11   97.142586        NaN         NaN
2010-01-12   96.236595        NaN         NaN
2010-01-13   97.049469        NaN         NaN
2010-01-14   97.311928        NaN         NaN
2010-01-15   96.219643        NaN         NaN
2010-01-19   97.421989        NaN         NaN
2010-01-20   96.431366        NaN         NaN
2010-01-21   94.577072        NaN         NaN
2010-01-22   92.468773        NaN         NaN
2010-01-25   92.942909        NaN         NaN
2010-01-26   92.553436        NaN         NaN
2010-01-27   92.993713        NaN         NaN
2010-01-28   91.926857        NaN         NaN
2010-01-29   90.927734        NaN         NaN
2010-02-01   92.341759        NaN 

                   SPY   Adj Close stockTicker
Date                                          
2010-01-04   95.957184         NaN         NaN
2010-01-05   96.211189         NaN         NaN
2010-01-06   96.278938         NaN         NaN
2010-01-07   96.685364         NaN         NaN
2010-01-08   97.007088         NaN         NaN
2010-01-11   97.142586         NaN         NaN
2010-01-12   96.236595         NaN         NaN
2010-01-13   97.049469         NaN         NaN
2010-01-14   97.311928         NaN         NaN
2010-01-15   96.219643         NaN         NaN
2010-01-19   97.421989         NaN         NaN
2010-01-20   96.431366         NaN         NaN
2010-01-21   94.577072         NaN         NaN
2010-01-22   92.468773         NaN         NaN
2010-01-25   92.942909         NaN         NaN
2010-01-26   92.553436         NaN         NaN
2010-01-27   92.993713         NaN         NaN
2010-01-28   91.926857         NaN         NaN
2010-01-29   90.927734         NaN         NaN
2010-02-01   

                   SPY  Adj Close stockTicker
Date                                         
2010-01-04   95.957184        NaN         NaN
2010-01-05   96.211189        NaN         NaN
2010-01-06   96.278938        NaN         NaN
2010-01-07   96.685364        NaN         NaN
2010-01-08   97.007088        NaN         NaN
2010-01-11   97.142586        NaN         NaN
2010-01-12   96.236595        NaN         NaN
2010-01-13   97.049469        NaN         NaN
2010-01-14   97.311928        NaN         NaN
2010-01-15   96.219643        NaN         NaN
2010-01-19   97.421989        NaN         NaN
2010-01-20   96.431366        NaN         NaN
2010-01-21   94.577072        NaN         NaN
2010-01-22   92.468773        NaN         NaN
2010-01-25   92.942909        NaN         NaN
2010-01-26   92.553436        NaN         NaN
2010-01-27   92.993713        NaN         NaN
2010-01-28   91.926857        NaN         NaN
2010-01-29   90.927734        NaN         NaN
2010-02-01   92.341759        NaN 

                   SPY  Adj Close stockTicker
Date                                         
2010-01-04   95.957184        NaN         NaN
2010-01-05   96.211189        NaN         NaN
2010-01-06   96.278938        NaN         NaN
2010-01-07   96.685364        NaN         NaN
2010-01-08   97.007088        NaN         NaN
2010-01-11   97.142586        NaN         NaN
2010-01-12   96.236595        NaN         NaN
2010-01-13   97.049469        NaN         NaN
2010-01-14   97.311928        NaN         NaN
2010-01-15   96.219643        NaN         NaN
2010-01-19   97.421989        NaN         NaN
2010-01-20   96.431366        NaN         NaN
2010-01-21   94.577072        NaN         NaN
2010-01-22   92.468773        NaN         NaN
2010-01-25   92.942909        NaN         NaN
2010-01-26   92.553436        NaN         NaN
2010-01-27   92.993713        NaN         NaN
2010-01-28   91.926857        NaN         NaN
2010-01-29   90.927734        NaN         NaN
2010-02-01   92.341759        NaN 

...

                   SPY  Adj Close stockTicker
Date                                         
2010-01-04   95.957184        NaN         NaN
2010-01-05   96.211189        NaN         NaN
2010-01-06   96.278938        NaN         NaN
2010-01-07   96.685364        NaN         NaN
2010-01-08   97.007088        NaN         NaN
2010-01-11   97.142586        NaN         NaN
2010-01-12   96.236595        NaN         NaN
2010-01-13   97.049469        NaN         NaN
2010-01-14   97.311928        NaN         NaN
2010-01-15   96.219643        NaN         NaN
2010-01-19   97.421989        NaN         NaN
2010-01-20   96.431366        NaN         NaN
2010-01-21   94.577072        NaN         NaN
2010-01-22   92.468773        NaN         NaN
2010-01-25   92.942909        NaN         NaN
2010-01-26   92.553436        NaN         NaN
2010-01-27   92.993713        NaN         NaN
2010-01-28   91.926857        NaN         NaN
2010-01-29   90.927734        NaN         NaN
2010-02-01   92.341759        NaN 

                   SPY  Adj Close stockTicker
Date                                         
2010-01-04   95.957184        NaN         NaN
2010-01-05   96.211189        NaN         NaN
2010-01-06   96.278938        NaN         NaN
2010-01-07   96.685364        NaN         NaN
2010-01-08   97.007088        NaN         NaN
2010-01-11   97.142586        NaN         NaN
2010-01-12   96.236595        NaN         NaN
2010-01-13   97.049469        NaN         NaN
2010-01-14   97.311928        NaN         NaN
2010-01-15   96.219643        NaN         NaN
2010-01-19   97.421989        NaN         NaN
2010-01-20   96.431366        NaN         NaN
2010-01-21   94.577072        NaN         NaN
2010-01-22   92.468773        NaN         NaN
2010-01-25   92.942909        NaN         NaN
2010-01-26   92.553436        NaN         NaN
2010-01-27   92.993713        NaN         NaN
2010-01-28   91.926857        NaN         NaN
2010-01-29   90.927734        NaN         NaN
2010-02-01   92.341759        NaN 

                   SPY   Adj Close stockTicker
Date                                          
2010-01-04   95.957184         NaN         NaN
2010-01-05   96.211189         NaN         NaN
2010-01-06   96.278938         NaN         NaN
2010-01-07   96.685364         NaN         NaN
2010-01-08   97.007088         NaN         NaN
2010-01-11   97.142586         NaN         NaN
2010-01-12   96.236595         NaN         NaN
2010-01-13   97.049469         NaN         NaN
2010-01-14   97.311928         NaN         NaN
2010-01-15   96.219643         NaN         NaN
2010-01-19   97.421989         NaN         NaN
2010-01-20   96.431366         NaN         NaN
2010-01-21   94.577072         NaN         NaN
2010-01-22   92.468773         NaN         NaN
2010-01-25   92.942909         NaN         NaN
2010-01-26   92.553436         NaN         NaN
2010-01-27   92.993713         NaN         NaN
2010-01-28   91.926857         NaN         NaN
2010-01-29   90.927734         NaN         NaN
2010-02-01   

                   SPY  Adj Close stockTicker
Date                                         
2010-01-04   95.957184        NaN         NaN
2010-01-05   96.211189        NaN         NaN
2010-01-06   96.278938        NaN         NaN
2010-01-07   96.685364        NaN         NaN
2010-01-08   97.007088        NaN         NaN
2010-01-11   97.142586        NaN         NaN
2010-01-12   96.236595        NaN         NaN
2010-01-13   97.049469        NaN         NaN
2010-01-14   97.311928        NaN         NaN
2010-01-15   96.219643        NaN         NaN
2010-01-19   97.421989        NaN         NaN
2010-01-20   96.431366        NaN         NaN
2010-01-21   94.577072        NaN         NaN
2010-01-22   92.468773        NaN         NaN
2010-01-25   92.942909        NaN         NaN
2010-01-26   92.553436        NaN         NaN
2010-01-27   92.993713        NaN         NaN
2010-01-28   91.926857        NaN         NaN
2010-01-29   90.927734        NaN         NaN
2010-02-01   92.341759        NaN 

                   SPY   Adj Close stockTicker
Date                                          
2010-01-04   95.957184         NaN         NaN
2010-01-05   96.211189         NaN         NaN
2010-01-06   96.278938         NaN         NaN
2010-01-07   96.685364         NaN         NaN
2010-01-08   97.007088         NaN         NaN
2010-01-11   97.142586         NaN         NaN
2010-01-12   96.236595         NaN         NaN
2010-01-13   97.049469         NaN         NaN
2010-01-14   97.311928         NaN         NaN
2010-01-15   96.219643         NaN         NaN
2010-01-19   97.421989         NaN         NaN
2010-01-20   96.431366         NaN         NaN
2010-01-21   94.577072         NaN         NaN
2010-01-22   92.468773         NaN         NaN
2010-01-25   92.942909         NaN         NaN
2010-01-26   92.553436         NaN         NaN
2010-01-27   92.993713         NaN         NaN
2010-01-28   91.926857         NaN         NaN
2010-01-29   90.927734         NaN         NaN
2010-02-01   

In [21]:
all_snp_price_hist_copy.loc[all_snp_price_hist_copy['stockTicker']=='MMM'].join(spy_dates)

,Adj Close,stockTicker,SPY
Date,,,
2012-02-09,75.432663,MMM,119.311707
2012-02-10,74.678513,MMM,118.430244
2012-02-13,75.441238,MMM,119.311707
2012-02-14,75.406952,MMM,119.161835
2012-02-15,75.070480,MMM,118.606567
2012-02-16,75.622635,MMM,119.919884
2012-02-17,75.544983,MMM,120.237221
2012-02-21,75.579506,MMM,120.290062
2012-02-22,75.717552,MMM,119.902267


In [22]:
#stock_price_merge_grade['firm'] = stock_price_merge_grade['firm'].str.replace('/','')

In [23]:
stock_price_merge_grade['firm'].unique()

array([u'CS First Boston', u'SBC Wrbg Dlln Rd', u'CIBC Oppenheimer',
       u'Furman Selz', u'Spencer Clarke', u'Goldman Sachs',
       u'Prudential Sec', u'Salomon Smth Brny', u'DLJ', u'AG Edwards',
       u'Lehman Bros', u'Mgn Stnly Dn Witter', u'Roney Capital Mkts',
       u'Schroder & Co', u'BT Alex. Brown', u'Bear Stearns',
       u'Merrill Lynch', u'PaineWebber', u'JP Morgan',
       u'Mrgn Stnly Dn Wttr', u'Adams Harkness', u'Piper Jaffray',
       u'Warburg Dlln Rd', u'BB Rbrtsn Stphns', u'ING Brng Frmn Selz',
       u'Friedman Billings', u'McDonald Invst', u'Deutsche Bank',
       u'ING Barings', u'Raymond James', u'ABN AMRO', u'USB Piper Jaffray',
       u'CSFB', u'Banc of America Sec', u'Brown Brothers', u'Prudential',
       u'Wasserstein Perella', u'Pacific Crest', u'Lehman Brothers',
       u'Warburg Dillon Read', u'Deutsche Bc  Alex. Br',
       u'H.C. Wainwright', u'Dresdner Klnwrt Bnsn', u'Sanford Bernstein',
       u'UBS Warburg', u'Jefferies & Co', u'Janney Mntgmy Sc

In [24]:
def dir_to_symbol(symbol,base_dir):
    return os.path.join(base_dir,'{}.csv'.format(symbol))  

def dir_to_path(path,base_dir='firm_grades'):
    return os.path.join(base_dir,'{}'.format(path))  


def make_dir(path):
    if not os.path.exists(path):
        return os.makedirs(path)

def merge_on_spy_dates(df,dates):
    '''
    makes folder called 'firm_grades'
    '''
    firms = []
    if not os.path.exists('firm_grades'):
        os.makedirs('firm_grades')
        '''
        1) checks dataframe for unique values of firm, creates path and cds to firm_grades/'firm'
        2) creates separate dataframe for all instances of firm upgrades
        '''
  
    for firm in df['firm'].unique():
        path = dir_to_path(firm,base_dir='firm_grades')
        
        
        make_dir(path)
        df1 = df.loc[df['firm']==firm]
        '''
        1)start an empty list
        2)for each of the stock tickers in the firm df
        3)in the path of firm_grade/'firm'/
        4)locate all of the unique values of the stocktickers that the
        firm upgraded or downgraded
        5)go to the main large db and locate all data for that symbol
        '''
        temp_list = []
        for symbol in df1['stockTicker'].unique():
#             temp_path=dir_to_symbol(symbol,base_dir=path)
            temp_df = df1.loc[df1['stockTicker']==symbol]
            dates_df = dates.loc[dates['stockTicker']==symbol]

                #df_dates = df_dates.join(temp_df)
            dates_df=dates_df.set_index(['stockTicker','Adj Close'],append=True).join(temp_df.set_index(['stockTicker','Adj Close'],append=True)).reset_index(level=['stockTicker','Adj Close'])
#             dates_df.to_csv(temp_path)
            temp_list.append(dates_df)
        firms.append(pd.concat(temp_list))
        print firm
    return pd.concat(firms)

In [25]:
final_df = merge_on_spy_dates(stock_price_merge_grade,spy_dates_symb)

CS First Boston
SBC Wrbg Dlln Rd
CIBC Oppenheimer
Furman Selz
Spencer Clarke
Goldman Sachs
Prudential Sec
Salomon Smth Brny
DLJ
AG Edwards
Lehman Bros
Mgn Stnly Dn Witter
Roney Capital Mkts
Schroder & Co
BT Alex. Brown
Bear Stearns
Merrill Lynch
PaineWebber
JP Morgan
Mrgn Stnly Dn Wttr
Adams Harkness
Piper Jaffray
Warburg Dlln Rd
BB Rbrtsn Stphns
ING Brng Frmn Selz
Friedman Billings
McDonald Invst
Deutsche Bank
ING Barings
Raymond James
ABN AMRO
USB Piper Jaffray
CSFB
Banc of America Sec
Brown Brothers
Prudential
Wasserstein Perella
Pacific Crest
Lehman Brothers
Warburg Dillon Read
Deutsche Bc  Alex. Br
H.C. Wainwright
Dresdner Klnwrt Bnsn
Sanford Bernstein
UBS Warburg
Jefferies & Co
Janney Mntgmy Scott
Dresdner Klnwrt Wass
Robert W. Baird
First Union Sec
Fahnestock
Legg Mason
Credit Lyonnais Sec
Morgan Stanley
Williams Capital
HC Wainwright
Thomas Weisel
CIBC Wrld Mkts
Deutsche Securities
SunTrust Rbsn Humphrey
DA Davidson
Wachovia
Oppenheimer
Smith Barney Citigroup
Blaylock & Partner

In [26]:
final_df.to_csv('full_df.csv')

In [27]:
df_final = pd.read_csv('full_df.csv')

In [28]:
df_final = df_final.set_index('Date')

In [47]:


def dir_to_path(path,base_dir='firm_grades'):
    return os.path.join(base_dir,'{}'.format(path))  

def make_dir(path):
    if not os.path.exists(path):
        return os.makedirs(path)


def dir_to_symbol(symbol,base_dir):
    return os.path.join(base_dir,'{}.csv'.format(symbol))  

def channel_dirs(df,path='firm_grades'):
    df_cols = df[['stockTicker', 'Adj Close','SPY']]
    make_dir(path)
    for firm in df.firm.unique():
        df_temp = df.loc[df['firm']==firm]
        firm_path = dir_to_path(firm,path)
        make_dir(firm_path)
        for grade in df_temp.toGrade_combined.unique():
            grade_df = df_temp.loc[df_temp['toGrade_combined']==grade]
            grade_path = dir_to_path(grade,firm_path)
            make_dir(grade_path)
            for symbol in grade_df.stockTicker.unique():
                symb_df = df_cols.loc[df_cols['stockTicker']==symbol].set_index(['stockTicker','Adj Close','SPY'],append=True)
                grade_symb_df = grade_df.loc[grade_df['stockTicker']==symbol].set_index(['stockTicker','Adj Close','SPY'],append=True)
                grade_symb_df_date_index = grade_symb_df.reset_index().drop_duplicates().set_index('Date')
                grade_symb_df_date_index.index = pd.to_datetime(grade_symb_df_date_index.index).date
                grade_symb_df_combined = symb_df.join(grade_symb_df).reset_index().drop_duplicates().set_index('Date')
            
                for date in grade_symb_df_date_index.index.values:
                    start_date = date
                    bus90str = str(start_date+BDay(90))[:-9]
                    bus90 = dt.datetime.strptime(bus90str,'%Y-%m-%d').date()
                    end_date = bus90
                    grade_symb_df_combined.index = pd.to_datetime(grade_symb_df_combined.index).date

                    if end_date in grade_symb_df_combined.index.values:
                        temp_df = grade_symb_df_combined.copy()
                        start = temp_df.index.searchsorted(start_date)
                        end = temp_df.index.searchsorted(end_date)
                        temp = temp_df[start:end]
                        f_path = dir_to_symbol(symbol+str(start),grade_path)
                        print temp
                        temp.to_csv(f_path)

In [48]:
channel_dirs(df_final,path='firm_grades')

           stockTicker  Adj Close         SPY           firm toGrade_combined
2012-02-08         YUM  39.460499  119.161835  Goldman Sachs             Hold
2012-02-09         YUM  39.748299  119.311707            NaN              NaN
2012-02-10         YUM  39.644199  118.430244            NaN              NaN
2012-02-13         YUM  39.546227  119.311707            NaN              NaN
2012-02-14         YUM  39.178810  119.161835            NaN              NaN
2012-02-15         YUM  39.019604  118.606567            NaN              NaN
2012-02-16         YUM  39.638073  119.919884            NaN              NaN
2012-02-17         YUM  39.956512  120.237221            NaN              NaN
2012-02-21         YUM  40.109596  120.290062            NaN              NaN
2012-02-22         YUM  40.354542  119.902267            NaN              NaN
2012-02-23         YUM  40.311684  120.431122            NaN              NaN
2012-02-24         YUM  40.134090  120.695549            NaN    

           stockTicker  Adj Close         SPY           firm toGrade_combined
2013-05-20         EFX  58.551029  151.070374  Goldman Sachs             Hold
2013-05-21         EFX  58.917690  151.287613            NaN              NaN
2013-05-22         EFX  58.115765  150.165451            NaN              NaN
2013-05-23         EFX  57.728954  149.731033            NaN              NaN
2013-05-24         EFX  57.747826  149.604340            NaN              NaN
2013-05-28         EFX  58.172375  150.500290            NaN              NaN
2013-05-29         EFX  57.983688  149.522858            NaN              NaN
2013-05-30         EFX  58.172375  150.074936            NaN              NaN
2013-05-31         EFX  57.455357  147.921036            NaN              NaN
2013-06-03         EFX  57.823303  148.735535            NaN              NaN
2013-06-04         EFX  57.474228  148.020569            NaN              NaN
2013-06-05         EFX  56.408142  145.948212            NaN    

           stockTicker  Adj Close         SPY           firm toGrade_combined
2012-02-13          HP  48.451763  119.311707  Goldman Sachs             Hold
2012-02-14          HP  48.272034  119.161835            NaN              NaN
2012-02-15          HP  48.059631  118.606567            NaN              NaN
2012-02-16          HP  49.276855  119.919884            NaN              NaN
2012-02-17          HP  51.694950  120.237221            NaN              NaN
2012-02-21          HP  50.878029  120.290062            NaN              NaN
2012-02-22          HP  51.547897  119.902267            NaN              NaN
2012-02-23          HP  51.221134  120.431122            NaN              NaN
2012-02-24          HP  51.237480  120.695549            NaN              NaN
2012-02-27          HP  50.779991  120.898293            NaN              NaN
2012-02-28          HP  50.134624  121.250877            NaN              NaN
2012-02-29          HP  50.077442  120.774857            NaN    

           stockTicker  Adj Close         SPY           firm toGrade_combined
2013-06-20         DIS  57.938595  144.255798  Goldman Sachs             Hold
2013-06-21         DIS  58.639694  144.718918            NaN              NaN
2013-06-24         DIS  58.368607  142.890228            NaN              NaN
2013-06-25         DIS  58.490128  144.264008            NaN              NaN
2013-06-26         DIS  59.088398  145.692368            NaN              NaN
2013-06-27         DIS  59.565144  146.547562            NaN              NaN
2013-06-28         DIS  59.032310  145.947083            NaN              NaN
2013-07-01         DIS  59.761452  146.802307            NaN              NaN
2013-07-02         DIS  59.135136  146.665848            NaN              NaN
2013-07-03         DIS  59.462315  146.729492            NaN              NaN
2013-07-05         DIS  59.658627  148.312531            NaN              NaN
2013-07-08         DIS  60.490593  149.158585            NaN    

           stockTicker  Adj Close         SPY           firm toGrade_combined
2013-06-21        NWSA  14.917256  144.718918  Goldman Sachs             Hold
2013-06-24        NWSA  14.831305  142.890228            NaN              NaN
2013-06-25        NWSA  15.012757  144.264008            NaN              NaN
2013-06-26        NWSA  14.707154  145.692368            NaN              NaN
2013-06-27        NWSA  14.802655  146.547562            NaN              NaN
2013-06-28        NWSA  14.563901  145.947083            NaN              NaN
2013-07-01        NWSA  14.124599  146.802307            NaN              NaN
2013-07-02        NWSA  14.325150  146.665848            NaN              NaN
2013-07-03        NWSA  14.200998  146.729492            NaN              NaN
2013-07-05        NWSA  14.955457  148.312531            NaN              NaN
2013-07-08        NWSA  15.241960  149.158585            NaN              NaN
2013-07-09        NWSA  15.003207  150.232224            NaN    

...

           stockTicker  Adj Close         SPY           firm toGrade_combined
2015-03-09         COP  56.581066  195.262100  Goldman Sachs             Hold
2015-03-10         COP  55.576721  192.094559            NaN              NaN
2015-03-11         COP  55.969330  191.644760            NaN              NaN
2015-03-12         COP  56.024113  194.081299            NaN              NaN
2015-03-13         COP  56.279762  192.891190            NaN              NaN
2015-03-16         COP  56.973671  195.468262            NaN              NaN
2015-03-17         COP  56.617588  194.887268            NaN              NaN
2015-03-18         COP  58.105846  197.230103            NaN              NaN
2015-03-19         COP  57.302368  196.330460            NaN              NaN
2015-03-20         COP  58.288448  198.063431            NaN              NaN
2015-03-23         COP  57.731499  197.677475            NaN              NaN
2015-03-24         COP  57.110626  196.566757            NaN    

           stockTicker  Adj Close         SPY           firm toGrade_combined
2017-12-04         HIG  56.227894  261.748413  Goldman Sachs             Hold
2017-12-05         HIG  54.376850  260.807007            NaN              NaN
2017-12-06         HIG  54.655502  260.856567            NaN              NaN
2017-12-07         HIG  54.506226  261.679047            NaN              NaN
2017-12-08         HIG  54.904297  263.106018            NaN              NaN
2017-12-11         HIG  55.153091  263.898743            NaN              NaN
2017-12-12         HIG  55.760155  264.364502            NaN              NaN
2017-12-13         HIG  55.282467  264.334778            NaN              NaN
2017-12-14         HIG  54.944103  263.254639            NaN              NaN
2017-12-15         HIG  55.481506  265.446869            NaN              NaN
2017-12-18         HIG  55.372036  267.130127            NaN              NaN
2017-12-19         HIG  56.456783  266.104248            NaN    

           stockTicker   Adj Close         SPY           firm toGrade_combined
2015-03-27        EQIX  207.234543  193.667480  Goldman Sachs             Hold
2015-03-30        EQIX  211.284912  196.030151            NaN              NaN
2015-03-31        EQIX  210.049026  194.316971            NaN              NaN
2015-04-01        EQIX  210.716568  193.629807            NaN              NaN
2015-04-02        EQIX  212.529785  194.326416            NaN              NaN
2015-04-06        EQIX  212.123810  195.634811            NaN              NaN
2015-04-07        EQIX  211.348068  195.117081            NaN              NaN
2015-04-08        EQIX  215.678009  195.776016            NaN              NaN
2015-04-09        EQIX  216.553055  196.642029            NaN              NaN
2015-04-10        EQIX  216.183212  197.715118            NaN              NaN
2015-04-13        EQIX  216.291428  196.820892            NaN              NaN
2015-04-14        EQIX  217.843018  197.197418      

           stockTicker  Adj Close         SPY           firm toGrade_combined
2015-04-13         CNP  17.934471  196.820892  Goldman Sachs             Hold
2015-04-14         CNP  18.074041  197.197418            NaN              NaN
2015-04-15         CNP  18.588699  198.082245            NaN              NaN
2015-04-16         CNP  18.196163  198.025757            NaN              NaN
2015-04-17         CNP  18.231056  195.747757            NaN              NaN
2015-04-20         CNP  18.422962  197.536270            NaN              NaN
2015-04-21         CNP  18.021702  197.300964            NaN              NaN
2015-04-22         CNP  18.169992  198.270508            NaN              NaN
2015-04-23         CNP  18.292116  198.769424            NaN              NaN
2015-04-24         CNP  18.361900  199.230667            NaN              NaN
2015-04-27         CNP  18.152550  198.402313            NaN              NaN
2015-04-28         CNP  18.327009  199.033020            NaN    

KeyboardInterrupt: 